# Iteration with Pandas

## [Dr. Tirthajyoti Sarkar](https://www.linkedin.com/in/tirthajyoti-sarkar-2127aa7/), Fremont, CA

As data scientists, all of us have been there.

We are given a large Pandas DataFrame and asked to check some relationships between various fields in the columns - in a row-by-row fashion. It could be some logical operation or some sophisticated mathematical transformation on the raw data.

Essentially, it is a **simple case of iterating over the rows of the DataFrame** and doing some processing at each iteration.

However, it **may not be that simple in terms of choosing the most efficient method** of executing this type of task. You might already have a hunch that a simple for-loop is going to be quite inefficient for this iteration task. 

And, you will be absolutely right. So, what can we do to avoid such speed pitfall?

This type of knowledge is critical to practice what we call ["Productive Data Science"](https://medium.com/productive-data-science/why-and-how-should-you-learn-productive-data-science-53377b473f37).

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from tqdm import tqdm
from time import time

In [2]:
np.random.seed(101)
df = pd.DataFrame(np.random.randint(0,100,size=(100000, 4)), 
                  columns=list('ABCD'),dtype=np.int16)
df.head()

,A,B,C,D
0,95,11,81,70
1,63,87,75,9
2,77,40,4,63
3,40,60,92,64
4,5,12,93,40


In [3]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   A       100000 non-null  int16
 1   B       100000 non-null  int16
 2   C       100000 non-null  int16
 3   D       100000 non-null  int16
dtypes: int16(4)
memory usage: 781.4 KB


In [4]:
df.describe()

,A,B,C,D
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,49.562640,49.533820,49.477140,49.495560
std,28.885105,28.907816,28.789594,28.849037
min,0.000000,0.000000,0.000000,0.000000
25%,25.000000,24.000000,25.000000,24.000000
50%,50.000000,50.000000,50.000000,50.000000
75%,75.000000,75.000000,74.000000,74.000000
max,99.000000,99.000000,99.000000,99.000000


## Most inefficient _for_ loop

In [5]:
count=0
t1=time()
for i in range(len(df)):
    if df.iloc[i]['A']+df.iloc[i]['B'] > df.iloc[i]['C']+df.iloc[i]['D']:
        count+=1
t2=time()
delt1 = round((t2-t1),2)
print(f"Time:{delt1} seconds")
print("Count:",count)

Time:32.22 seconds
Count: 49769


## Comparing `iterrows()` and `df.values` - 1

In [6]:
count=0
t1=time()
for idx, row in df.iterrows():
    if row['A']+row['B'] > (row['C']+row['D']):
        count+=1
t2=time()
delt1 = round((t2-t1),2)
print(f"Time:{delt1} seconds")
print("Count:",count)

Time:6.91 seconds
Count: 49769


In [7]:
count=0
t1=time()
for row in df.values:
    if row[0]+row[1] > (row[2]+row[3]):
        count+=1
t2=time()
delt2 = round((t2-t1),3)
print(f"Time:{delt2} seconds")
print("Count:",count)

Time:0.112 seconds
Count: 49769


In [8]:
print(f"df.values is {round(delt1/delt2,2)} times faster")

df.values is 61.7 times faster


## Comparing `iterrows()` and `df.values` - 2

In [9]:
count=0
t1=time()
for idx, row in df.iterrows():
    if row['A']+row['B'] > 1.25*(row['C']+row['D']):
        count+=1
t2=time()
delt1 = round((t2-t1),2)
print(f"Time:{delt1} seconds")
print("Count:",count)

Time:8.05 seconds
Count: 35886


In [10]:
count=0
t1=time()
for row in df.values:
    if row[0]+row[1] > 1.25*(row[2]+row[3]):
        count+=1
t2=time()
delt2 = round((t2-t1),3)
print(f"Time:{delt2} seconds")
print("Count:",count)

Time:0.546 seconds
Count: 35886


In [11]:
print(f"df.values is {round(delt1/delt2,2)} times faster")

df.values is 14.74 times faster


## Comparing `iterrows()` and `df.values` - 3

In [12]:
count=0
t1=time()
for idx, row in df.iterrows():
    if np.log(1+row['A']+row['B']) > np.sqrt(0.5*(row['C']+row['D'])):
        count+=1
t2=time()
delt1 = round((t2-t1),2)
print(f"Time:{delt1} seconds")
print("Count:",count)

Time:8.76 seconds
Count: 9202


In [13]:
count=0
t1=time()
for row in df.values:
    if np.log(1+row[0]+row[1]) > np.sqrt(0.5*(row[2]+row[3])):
        count+=1
t2=time()
delt2 = round((t2-t1),3)
print(f"Time:{delt2} seconds")
print("Count:",count)

Time:0.962 seconds
Count: 9202


In [14]:
print(f"df.values is {round(delt1/delt2,2)} times faster")

df.values is 9.11 times faster


## Simple vectorized operation is fastest in this counting example

In [15]:
t1 = time()
df['result'] = np.log(1+df['A']+df['B']) > np.sqrt(0.5*(df['C']+df['D']))
t2=time()
delt3 = round((t2-t1),3)
print(f"Time:{delt3} seconds")
print("Count:",df['result'].sum())

Time:0.01 seconds
Count: 9202


## String identifier

In [16]:
def identifier():
    """
    Generates random identifier string of 5 characters
    """
    letters = list('CFJQZ')
    numbers = list('123456789')
    
    random_id = ''
    random_id+=np.random.choice(letters)
    random_id+=np.random.choice(letters)
    random_id+=np.random.choice(numbers)
    random_id+=np.random.choice(numbers)
    #random_id+=np.random.choice(numbers)
    #random_id+=np.random.choice(numbers)
    #random_id+=np.random.choice(letters)
    random_id+=np.random.choice(letters)
    
    return random_id

In [17]:
for i in range(10):
    print(identifier())

JQ98C
CC24Z
ZQ25Q
QC54C
ZZ24Z
JF91Z
FQ89C
FQ15F
CZ59F
ZC59F


In [18]:
df.head()

,A,B,C,D,result
0,95,11,81,70,False
1,63,87,75,9,False
2,77,40,4,63,False
3,40,60,92,64,False
4,5,12,93,40,False


In [19]:
id_lst=[]
for i in range(100000):
    id_lst.append(identifier())
id_lst=np.array(id_lst)

df.insert(0,'ID',id_lst)

In [20]:
df.sample(5)

,ID,A,B,C,D,result
78891,ZF36F,19,64,85,67,False
6152,FC69F,21,2,22,12,False
99247,FC76F,3,77,78,32,False
63451,ZF94Q,30,95,10,12,True
40941,QJ46F,14,4,95,40,False


In [21]:
df['ID'].nunique()

10125

In [22]:
ratio_dict={'ID':[],'Ratio':[]}
t1=time()
for _,row in df.iterrows():
    if row['ID'][0:2] == 'ZZ' and row['ID'][-1]=='F':
        ratio = (row['A']+row['B'])/(0.01+row['C']+row['D'])
        ratio_dict['ID'].append(row[0])
        ratio_dict['Ratio'].append(ratio)
t2=time()
delt4 = round((t2-t1),3)
print(f"Time:{delt4} seconds")

Time:6.597 seconds


In [23]:
ratio_dict={'ID':[],'Ratio':[]}
t1=time()
for row in df.values:
    if row[0][0:2] == 'ZZ' and row[0][-1]=='F':
        ratio = (row[1]+row[2])/(0.01+row[3]+row[4])
        ratio_dict['ID'].append(row[0])
        ratio_dict['Ratio'].append(ratio)
t2=time()
delt4 = round((t2-t1),3)
print(f"Time:{delt4} seconds")

Time:0.056 seconds


In [24]:
pd.DataFrame(ratio_dict)

,ID,Ratio
0,ZZ43F,0.349377
1,ZZ79F,1.285600
2,ZZ57F,1.090744
3,ZZ17F,0.862388
4,ZZ56F,1.134152
...,...,...
822,ZZ34F,0.888724
823,ZZ22F,0.767811
824,ZZ59F,2.063838
825,ZZ47F,0.608283
